# PySpark ile bir Verisetindeki Mesleklerin ortalama maaşını Hesaplama
## **mapValues** ve **reduceByKey** metodlarının kullanımı

### *1* - Gerekli Kütüphanelerin import edilmesi ve verisetinin alınması

In [1]:
import findspark
findspark.init()

In [3]:
from pyspark import SparkContext

In [4]:
sc=SparkContext("local[4]","PairRDD-Ops")

In [5]:
insanlarRDD2=sc.textFile("simple_data.csv")

In [7]:
insanlarRDD2.take(4)

['sirano,isim,yas,meslek,sehir,aylik_gelir',
 '1,Cemal,35,Isci,Ankara,3500',
 '2,Ceyda,42,Memur,Kayseri,4200',
 '3,Timur,30,Müzisyen,Istanbul,9000']

### *2* - Verisetinin ilk satırının çıkarılması

In [9]:
insanlarRDD=insanlarRDD2.filter(lambda x: "sirano" not in x)

In [10]:
insanlarRDD.take(4)

['1,Cemal,35,Isci,Ankara,3500',
 '2,Ceyda,42,Memur,Kayseri,4200',
 '3,Timur,30,Müzisyen,Istanbul,9000',
 '4,Burcu,29,Pazarlamaci,Ankara,4200']

### *3* - Meslek ve maaş kolonlarının alınması

In [11]:
def meslek_mass(line):
    meslek=line.split(",")[3]
    maas=float(line.split(",")[5])
    return (meslek,maas)

### *4* - Metodun verisetine uygulanması

In [22]:
meslek_maas_pairRDD= insanlarRDD.map(meslek_mass)

In [23]:
meslek_maas_pairRDD.take(8)

[('Isci', 3500.0),
 ('Memur', 4200.0),
 ('Müzisyen', 9000.0),
 ('Pazarlamaci', 4200.0),
 ('Pazarlamaci', 4800.0),
 ('Memur', 4250.0),
 ('Pazarlamaci', 7300.0),
 ('Müzisyen', 12000.0)]

### *5* - mapValues ile maaşların mesleğe göre toplanması için (maas,1) formatında tuple oluşturma

In [14]:
meslek_maas=meslek_maas_pairRDD.mapValues(lambda x:(x,1))

In [24]:
meslek_maas.take(9)

[('Isci', (3500.0, 1)),
 ('Memur', (4200.0, 1)),
 ('Müzisyen', (9000.0, 1)),
 ('Pazarlamaci', (4200.0, 1)),
 ('Pazarlamaci', (4800.0, 1)),
 ('Memur', (4250.0, 1)),
 ('Pazarlamaci', (7300.0, 1)),
 ('Müzisyen', (12000.0, 1)),
 ('Doktor', (18000.0, 1))]

### *6* - reduceByKey ile mesleğe göre maaşların toplanması ve kaç verinin toplandığını belirtme

In [18]:
meslek_maas_RBK=meslek_maas.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1]))

In [19]:
meslek_maas_RBK.take(10)

[('Memur', (12200.0, 3)),
 ('Pazarlamaci', (16300.0, 3)),
 ('Tuhafiyeci', (4800.0, 1)),
 ('Tornacı', (4200.0, 1)),
 ('Isci', (3500.0, 1)),
 ('Müzisyen', (29700.0, 3)),
 ('Doktor', (32250.0, 2)),
 ('Berber', (12000.0, 1))]

### *7* - mapValues ile maaşların ortalamasının alınması

In [21]:
meslek_ortalama_maas=meslek_maas_RBK.mapValues(lambda x:x[0]/x[1])
meslek_ortalama_maas.take(8)

[('Memur', 4066.6666666666665),
 ('Pazarlamaci', 5433.333333333333),
 ('Tuhafiyeci', 4800.0),
 ('Tornacı', 4200.0),
 ('Isci', 3500.0),
 ('Müzisyen', 9900.0),
 ('Doktor', 16125.0),
 ('Berber', 12000.0)]

# **Faydalı olması dileğiyle**